In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1_yOuaRupWcvvBB5tNnjVrtDllXqg6x4Q", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/03_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Setup Run Cell
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_01_setup_run_cell.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Drunk Hiker Analogy
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_03_drunk_hiker_analogy.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Why Both Terms Matter
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_04_why_both_terms_matter.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Update Rule
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_05_math_update_rule.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Sqrt Eta
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_06_sqrt_2_eta.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Numerical Example
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_07_numerical_example.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Code Walkthrough: Recreate Score Model
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_08_recreate_score_model.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# Langevin Dynamics: Sampling with Score Functions -- Vizuara

## 1. Why Does This Matter?

We have learned how to train a score function using Denoising Score Matching. But a score function alone does not generate data -- it only tells us which direction to move at any point in space.

**Langevin Dynamics** is the missing piece. It is a sampling algorithm that takes our trained score function and uses it to navigate from random noise to realistic data samples. It is the "engine" that powers generation in score-based models.

By the end of this notebook, you will:
- Understand Langevin Dynamics as a stochastic process guided by the score
- Implement Langevin sampling from scratch
- Visualize sampling trajectories from noise to data
- Generate new data points that match the training distribution
- Understand why the noise term in Langevin updates is critical

## 2. Building Intuition

### The Drunk Hiker Analogy

Imagine you are dropped into a thick fog on a vast landscape. Hidden somewhere are deep valleys (high-probability regions). You have a compass (the score function) that points downhill toward the nearest valley.

If you simply follow the compass perfectly (gradient descent), you might get stuck in a small pothole -- a local minimum that is not the true valley.

So instead, you follow the compass direction but also stumble randomly at each step. You are a "drunk hiker." This random stumbling lets you escape shallow potholes and eventually find the true deep valleys.

The update rule at each step:
$$x_{t+1} = x_t + \underbrace{\eta \cdot s_\theta(x_t)}_{\text{follow compass}} + \underbrace{\sqrt{2\eta} \cdot \epsilon_t}_{\text{random stumble}}$$

The first term pulls you toward high-probability regions. The second term adds randomness to help exploration.

### Why Both Terms Matter

- **Without the noise term:** Pure gradient ascent. Gets stuck in local modes. Deterministic -- always produces the same output from the same starting point.
- **Without the score term:** Pure random walk. Wanders aimlessly forever.
- **Together:** Theoretically guaranteed to sample from the correct distribution (given enough steps and small enough step size).

## 3. The Mathematics

### The Langevin Dynamics Update Rule

Starting from a random point $x_0 \sim \mathcal{N}(0, I)$, iterate:

$$x_{t+1} = x_t + \eta \cdot s_\theta(x_t) + \sqrt{2\eta} \cdot \epsilon_t, \quad \epsilon_t \sim \mathcal{N}(0, I)$$

where $\eta$ is the step size and $s_\theta$ is the learned score function.

**Computationally:** at each step, evaluate the score network, scale by step size, add scaled Gaussian noise, and update the position.

### Why sqrt(2 * eta)?

The specific scaling $\sqrt{2\eta}$ comes from the theory of Langevin diffusion. It ensures that as $\eta \to 0$ and the number of steps $\to \infty$, the samples converge to the true distribution. The factor of 2 ensures the correct balance between drift (score) and diffusion (noise).

### Numerical Example

Let us trace through 3 steps. Starting point: $x_0 = [4.0, -3.0]$, step size $\eta = 0.1$, and suppose the score function points toward the origin.

**Step 1:** $s_\theta(x_0) = [-1.5, 1.2]$, $\epsilon_0 = [0.3, -0.5]$

$$x_1 = [4.0, -3.0] + 0.1 \times [-1.5, 1.2] + \sqrt{0.2} \times [0.3, -0.5]$$
$$= [4.0, -3.0] + [-0.15, 0.12] + [0.134, -0.224]$$
$$= [3.984, -3.104]$$

**Step 2:** $s_\theta(x_1) = [-1.4, 1.3]$, $\epsilon_1 = [-0.1, 0.8]$

$$x_2 = [3.984, -3.104] + 0.1 \times [-1.4, 1.3] + \sqrt{0.2} \times [-0.1, 0.8]$$
$$= [3.984, -3.104] + [-0.14, 0.13] + [-0.045, 0.358]$$
$$= [3.799, -2.616]$$

**Step 3:** $s_\theta(x_2) = [-1.2, 1.1]$, $\epsilon_2 = [0.6, 0.2]$

$$x_3 = [3.799, -2.616] + 0.1 \times [-1.2, 1.1] + \sqrt{0.2} \times [0.6, 0.2]$$
$$= [3.799, -2.616] + [-0.12, 0.11] + [0.268, 0.089]$$
$$= [3.947, -2.417]$$

The point is gradually moving toward the data region, with a zigzag path due to the noise. Over many more steps, it will converge to a data cluster.

## 4. Let's Build It -- Component by Component

### 4.1 Re-create the DSM-trained Score Model

First, let us quickly re-train a score model using DSM (from the previous notebook).

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Generate data
def generate_data(n_samples=2000):
    cluster1 = torch.randn(n_samples // 2, 2) * 0.5 + torch.tensor([2.0, 2.0])
    cluster2 = torch.randn(n_samples // 2, 2) * 0.5 + torch.tensor([-2.0, -2.0])
    return torch.cat([cluster1, cluster2], dim=0)

class ScoreNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128), nn.SiLU(),
            nn.Linear(128, 128), nn.SiLU(),
            nn.Linear(128, 2)
        )
    def forward(self, x):
        return self.net(x)

# Train with DSM
data = generate_data(2000)
model = ScoreNetwork()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
sigma = 0.5

for epoch in range(2000):
    noise = torch.randn_like(data) * sigma
    noisy_data = data + noise
    target = (data - noisy_data) / (sigma ** 2)
    pred = model(noisy_data)
    loss = ((pred - target) ** 2).sum(dim=-1).mean()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f"Training complete. Final loss: {loss.item():.4f}")

In [ ]:
#@title 🎧 What to Look For: Visualize Score Field
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_09_visualize_score_field.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Verify: visualize the learned score field
n_grid = 20
x_range = torch.linspace(-5, 5, n_grid)
y_range = torch.linspace(-5, 5, n_grid)
xx, yy = torch.meshgrid(x_range, y_range, indexing='ij')
grid = torch.stack([xx.flatten(), yy.flatten()], dim=1)

with torch.no_grad():
    scores = model(grid)

plt.figure(figsize=(8, 8))
plt.scatter(data[:, 0], data[:, 1], alpha=0.1, s=3, c='blue')
plt.quiver(grid[:, 0], grid[:, 1], scores[:, 0], scores[:, 1],
           color='red', alpha=0.7, scale=50)
plt.title('Score Field (Our Compass)')
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.show()
print("Arrows point toward data clusters. Our compass is ready!")

In [ ]:
#@title 🎧 Transition: Basic Langevin Sampling Heading
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_10_basic_langevin_sampling_heading.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Basic Langevin Sampling

In [ ]:
#@title 🎧 Code Walkthrough: Langevin Sample Function
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_11_langevin_sample_function.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def langevin_sample(model, n_steps=1000, step_size=0.01,
                    n_samples=500, dim=2, record_every=50):
    """
    Generate samples using Langevin Dynamics.

    Args:
        model: trained score network
        n_steps: number of Langevin steps
        step_size: eta (learning rate for Langevin)
        n_samples: how many independent chains to run
        dim: data dimensionality
        record_every: save trajectory every N steps

    Returns:
        final_samples: (n_samples, dim) -- final positions
        trajectories: list of (n_samples, dim) tensors at intervals
    """
    # Start from random noise
    x = torch.randn(n_samples, dim) * 3.0
    trajectories = [x.clone()]

    for t in range(n_steps):
        with torch.no_grad():
            score = model(x)

        # Langevin update
        noise = torch.randn_like(x)
        x = x + step_size * score + np.sqrt(2 * step_size) * noise

        if (t + 1) % record_every == 0:
            trajectories.append(x.clone())

    return x, trajectories

# Generate samples
samples, trajectories = langevin_sample(model, n_steps=1000, step_size=0.01)

plt.figure(figsize=(8, 8))
plt.scatter(data[:, 0], data[:, 1], alpha=0.1, s=3, c='blue', label='True data')
plt.scatter(samples[:, 0], samples[:, 1], alpha=0.3, s=5, c='red', label='Generated')
plt.legend(fontsize=12)
plt.title('Langevin Dynamics: Generated vs True Data')
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Generated {len(samples)} samples using {1000} Langevin steps")
print(f"The red dots should overlap with the blue clusters!")

In [ ]:
#@title 🎧 Transition: Visualize Trajectories Heading
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_12_visualize_trajectories_heading.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Visualizing Trajectories

In [ ]:
#@title 🎧 What to Look For: Visualize Trajectories Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_13_visualize_trajectories_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Show individual sampling trajectories
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, ax in enumerate(axes):
    # Get one chain's trajectory
    chain_traj = torch.stack([t[idx] for t in trajectories]).numpy()

    # Plot data
    ax.scatter(data[:, 0], data[:, 1], alpha=0.05, s=2, c='blue')

    # Plot trajectory
    ax.plot(chain_traj[:, 0], chain_traj[:, 1], 'g.-', alpha=0.7,
            markersize=4, linewidth=1)
    ax.plot(chain_traj[0, 0], chain_traj[0, 1], 'go', markersize=10,
            label='Start', zorder=5)
    ax.plot(chain_traj[-1, 0], chain_traj[-1, 1], 'r*', markersize=15,
            label='End', zorder=5)

    ax.set_title(f'Trajectory {idx+1}', fontsize=13)
    ax.legend()
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)

plt.suptitle('Langevin Sampling Trajectories (Drunk Hiker Paths)', fontsize=15)
plt.tight_layout()
plt.show()

print("Each trajectory starts from noise (green) and zigzags toward data (red).")
print("The zigzag pattern is from the noise term -- our 'drunk hiker' stumbling.")

In [ ]:
#@title 🎧 Before You Start: Todo Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_14_todo_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn -- TODO Exercises

### TODO 1: Implement Langevin Sampling Without Noise

In [ ]:
#@title 🎧 Before You Start: Todo1 Implement Without Noise
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_15_todo1_implement_without_noise.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def deterministic_sample(model, n_steps=1000, step_size=0.01,
                         n_samples=500, dim=2):
    """
    Generate samples using pure gradient ascent (NO noise term).
    This is Langevin dynamics without the stochastic part.

    Args:
        model: trained score network
        n_steps: number of steps
        step_size: eta
        n_samples: number of chains
        dim: data dimensionality

    Returns:
        final_samples: (n_samples, dim)
    """
    x = torch.randn(n_samples, dim) * 3.0

    # ============ TODO ============
    # Implement the update WITHOUT the noise term:
    # x = x + step_size * score
    # (No sqrt(2*eta) * noise term!)
    # ==============================

    for t in range(n_steps):
        pass  # YOUR CODE HERE

    return x

In [ ]:
#@title 🎧 What to Look For: Todo1 Visualization
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_16_todo1_visualization.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Compare: with noise vs without noise
det_samples = deterministic_sample(model)

if det_samples is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    axes[0].scatter(data[:, 0], data[:, 1], alpha=0.1, s=3, c='blue')
    axes[0].scatter(samples[:, 0], samples[:, 1], alpha=0.3, s=5, c='red')
    axes[0].set_title('With Noise (Langevin)')
    axes[0].set_aspect('equal')
    axes[0].grid(True, alpha=0.3)

    axes[1].scatter(data[:, 0], data[:, 1], alpha=0.1, s=3, c='blue')
    axes[1].scatter(det_samples[:, 0], det_samples[:, 1], alpha=0.3, s=5, c='orange')
    axes[1].set_title('Without Noise (Deterministic)')
    axes[1].set_aspect('equal')
    axes[1].grid(True, alpha=0.3)

    plt.suptitle('Effect of Noise in Langevin Dynamics')
    plt.tight_layout()
    plt.show()

    print("Without noise: samples collapse to mode centers (less diverse)")
    print("With noise: samples spread naturally like the true distribution")

In [ ]:
#@title 🎧 Before You Start: Todo2 Effect Of Step Size Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_17_todo2_effect_of_step_size_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Effect of Step Size

In [ ]:
#@title 🎧 Before You Start: Todo2 Implement Step Sizes
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_18_todo2_implement_step_sizes.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def experiment_step_sizes(model, step_sizes, n_steps=1000, n_samples=300):
    """
    Run Langevin sampling with different step sizes and return results.

    Args:
        model: trained score model
        step_sizes: list of eta values to try
        n_steps: steps per chain
        n_samples: chains per experiment

    Returns:
        results: dict mapping step_size -> final samples tensor
    """
    results = {}

    # ============ TODO ============
    # For each step_size in step_sizes:
    #   1. Call langevin_sample with that step size
    #   2. Store the final samples in results[step_size]
    # ==============================

    return results

# step_sizes = [0.001, 0.01, 0.05, 0.2]
# results = experiment_step_sizes(model, step_sizes)
#
# fig, axes = plt.subplots(1, len(step_sizes), figsize=(6*len(step_sizes), 6))
# for ax, eta in zip(axes, step_sizes):
#     samps = results[eta]
#     ax.scatter(data[:, 0], data[:, 1], alpha=0.1, s=3, c='blue')
#     ax.scatter(samps[:, 0], samps[:, 1], alpha=0.3, s=5, c='red')
#     ax.set_title(f'eta = {eta}')
#     ax.set_aspect('equal')
#     ax.grid(True, alpha=0.3)
#     ax.set_xlim(-7, 7)
#     ax.set_ylim(-7, 7)
# plt.suptitle('Effect of Step Size on Langevin Sampling')
# plt.tight_layout()
# plt.show()

In [ ]:
#@title 🎧 Transition: Putting It All Together Heading
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_19_putting_it_all_together_heading.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

In [ ]:
#@title 🎧 Narration: Complete Pipeline
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_20_complete_pipeline.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Complete pipeline: Data -> DSM Training -> Langevin Sampling
print("=" * 60)
print("Complete Score-Based Generative Pipeline")
print("=" * 60)

# Step 1: Generate data (already done)
print(f"\n1. Training data: {len(data)} samples from 2-cluster mixture")

# Step 2: Train score model with DSM (already done)
print(f"2. Score model trained with DSM (sigma={sigma})")

# Step 3: Sample with Langevin dynamics
n_generated = 1000
final_samples, traj = langevin_sample(model, n_steps=2000,
                                       step_size=0.01,
                                       n_samples=n_generated,
                                       record_every=200)

print(f"3. Generated {n_generated} new samples via Langevin dynamics")

In [ ]:
#@title 🎧 What to Look For: Visualize Evolution
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_21_visualize_evolution.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize the evolution of samples over time
n_snapshots = len(traj)
fig, axes = plt.subplots(2, (n_snapshots + 1) // 2, figsize=(20, 10))
axes = axes.flatten()

for i, (ax, snapshot) in enumerate(zip(axes, traj)):
    ax.scatter(data[:, 0], data[:, 1], alpha=0.05, s=2, c='blue')
    ax.scatter(snapshot[:, 0], snapshot[:, 1], alpha=0.2, s=5, c='red')
    step_num = i * 200 if i > 0 else 0
    ax.set_title(f'Step {step_num}', fontsize=11)
    ax.set_xlim(-7, 7)
    ax.set_ylim(-7, 7)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.2)

# Hide extra axes
for j in range(len(traj), len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Evolution of Langevin Samples Over Time', fontsize=15)
plt.tight_layout()
plt.show()

print("Watch how samples start as random noise and gradually")
print("converge to the data distribution!")

In [ ]:
#@title 🎧 Transition: Training And Results Heading
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_22_training_and_results_heading.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

In [ ]:
#@title 🎧 What to Look For: Kde Comparison
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_23_kde_comparison.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Quantitative comparison: KDE of generated vs true
from scipy.stats import gaussian_kde

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# X1 marginal
for ax, dim_idx, dim_name in [(axes[0], 0, 'x1'), (axes[1], 1, 'x2')]:
    true_vals = data[:, dim_idx].numpy()
    gen_vals = final_samples[:, dim_idx].numpy()

    x_plot = np.linspace(-6, 6, 200)
    kde_true = gaussian_kde(true_vals)
    kde_gen = gaussian_kde(gen_vals)

    ax.plot(x_plot, kde_true(x_plot), 'b-', linewidth=2, label='True')
    ax.plot(x_plot, kde_gen(x_plot), 'r--', linewidth=2, label='Generated')
    ax.set_title(f'Marginal Distribution: {dim_name}', fontsize=13)
    ax.set_xlabel(dim_name)
    ax.set_ylabel('Density')
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)

plt.suptitle('Generated vs True Marginal Distributions', fontsize=15)
plt.tight_layout()
plt.show()

print("The red dashed line (generated) closely follows the blue line (true).")
print("Our score-based generative model works!")

In [ ]:
#@title 🎧 Transition: Final Output Heading
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_24_final_output_heading.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
#@title 🎧 What to Look For: Grand Finale Visualization
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_25_grand_finale_visualization.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Grand finale: complete visualization
fig = plt.figure(figsize=(20, 8))

# Panel 1: Training data
ax1 = fig.add_subplot(1, 3, 1)
ax1.scatter(data[:, 0], data[:, 1], alpha=0.3, s=5, c='blue')
ax1.set_title('1. Training Data', fontsize=14)
ax1.set_xlim(-6, 6)
ax1.set_ylim(-6, 6)
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)

# Panel 2: Learned score field
ax2 = fig.add_subplot(1, 3, 2)
with torch.no_grad():
    field_scores = model(grid)
ax2.scatter(data[:, 0], data[:, 1], alpha=0.1, s=3, c='blue')
ax2.quiver(grid[:, 0], grid[:, 1],
           field_scores[:, 0], field_scores[:, 1],
           color='red', alpha=0.7, scale=50)
ax2.set_title('2. Learned Score Field (DSM)', fontsize=14)
ax2.set_xlim(-6, 6)
ax2.set_ylim(-6, 6)
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)

# Panel 3: Generated samples
ax3 = fig.add_subplot(1, 3, 3)
ax3.scatter(data[:, 0], data[:, 1], alpha=0.15, s=5, c='blue', label='True')
ax3.scatter(final_samples[:, 0], final_samples[:, 1],
            alpha=0.3, s=5, c='red', label='Generated')
ax3.legend(fontsize=12)
ax3.set_title('3. Generated Samples (Langevin)', fontsize=14)
ax3.set_xlim(-6, 6)
ax3.set_ylim(-6, 6)
ax3.set_aspect('equal')
ax3.grid(True, alpha=0.3)

plt.suptitle('Score-Based Generative Modeling: The Full Pipeline', fontsize=16)
plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("SUMMARY: The Score-Based Generative Pipeline")
print("=" * 60)
print("1. Collect training data from the target distribution")
print("2. Train a score network using Denoising Score Matching")
print("   (add noise, predict direction back to clean data)")
print("3. Generate new samples using Langevin Dynamics")
print("   (follow the score compass with random stumbles)")
print("\nThis is the foundation of modern diffusion models!")
print("DDPM, Stable Diffusion, and DALL-E all build on this principle.")

In [ ]:
#@title 🎧 Listen: Reflection And Next Steps
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_26_reflection_and_next_steps.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### Think About This

1. **Why do we start from random noise?** What would happen if we started from a fixed point like [0, 0]?

2. **How many Langevin steps are "enough"?** What visual cues tell you that the chains have converged?

3. **What is the trade-off between step size and number of steps?** Can a large step size compensate for fewer steps?

4. **How does this relate to the reverse diffusion process in DDPM?** In DDPM, each denoising step is guided by a noise prediction network. How is that similar to a Langevin step guided by a score network?

### Extension Challenges

1. **Annealed Langevin Dynamics:** Instead of a fixed noise level, start with large noise (large sigma) and gradually decrease it. This helps the sampler first find the rough location of modes, then refine. Implement this and compare with fixed-sigma sampling.

2. **3D Data:** Generate data from a 3D mixture of Gaussians and visualize the sampling trajectories in 3D using plotly or matplotlib's 3D capabilities.

3. **Mode coverage:** With two clusters, does Langevin dynamics visit both modes? Run 100 chains and check what fraction end up in each cluster. Is it roughly 50/50?

### The Big Picture

You now understand the complete score-based generative modeling pipeline:

- **Score function:** The gradient of the log density, a vector field pointing toward data
- **Denoising Score Matching:** A tractable way to learn the score by predicting noise directions
- **Langevin Dynamics:** A stochastic sampling algorithm that uses the score to generate new data

Modern diffusion models (DDPM, Score SDE) are sophisticated extensions of these exact ideas, with multi-scale noise levels, U-Net architectures, and continuous-time formulations. But the core principle remains the same: learn the score, then sample with it.